# Front Page DeepExplainer MNIST

Source: [SHAP documentation](https://shap-lrjball.readthedocs.io/en/latest/example_notebooks/deep_explainer/Front%20Page%20DeepExplainer%20MNIST%20Example.html)

A simple example showing how to explain an MNIST CNN trained using Keras with DeepExplainer.

In [2]:
from __future__ import print_function

import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [3]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [6]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

In [7]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [8]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/12


2022-05-26 22:06:15.310844: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


469/469 [==============================] - 22s 46ms/step - loss: 2.2797 - accuracy: 0.1539 - val_loss: 2.2381 - val_accuracy: 0.3093
Epoch 2/12
469/469 [==============================] - 22s 46ms/step - loss: 2.2149 - accuracy: 0.2711 - val_loss: 2.1579 - val_accuracy: 0.4743
Epoch 3/12
469/469 [==============================] - 22s 47ms/step - loss: 2.1320 - accuracy: 0.3712 - val_loss: 2.0515 - val_accuracy: 0.5980
Epoch 4/12
469/469 [==============================] - 22s 47ms/step - loss: 2.0209 - accuracy: 0.4515 - val_loss: 1.9086 - val_accuracy: 0.6657
Epoch 5/12
469/469 [==============================] - 22s 48ms/step - loss: 1.8797 - accuracy: 0.5095 - val_loss: 1.7313 - val_accuracy: 0.6928
Epoch 6/12
469/469 [==============================] - 22s 48ms/step - loss: 1.7178 - accuracy: 0.5513 - val_loss: 1.5311 - val_accuracy: 0.7259
Epoch 7/12
469/469 [==============================] - 22s 48ms/step - loss: 1.5430 - accuracy: 0.5890 - val_loss: 1.3274 - val_accuracy: 0.7639
Epo

In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7179750800132751
Test accuracy: 0.838100016117096


In [10]:
import shap
import numpy as np

# select a set of background examples to take an expectation over
background = x_train[np.random.choice(x_train.shape[0], 100, replace=False)]

# explain predictions of the model on three images
e = shap.DeepExplainer(model, background)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(x_test[1:5])

keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


AttributeError: in user code:

    File "/opt/homebrew/Caskroom/miniforge/base/envs/tf2/lib/python3.10/site-packages/shap/explainers/_deep/deep_tf.py", line 243, in grad_graph  *
        out = self.model(shap_rAnD)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/tf2/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/opt/homebrew/Caskroom/miniforge/base/envs/tf2/lib/python3.10/site-packages/shap/explainers/_deep/deep_tf.py", line 26, in custom_record_gradient
        out = tf_backprop._record_gradient("shap_"+op_name, inputs, attrs, results)

    AttributeError: Exception encountered when calling layer "conv2d" (type Conv2D).
    
    module 'tensorflow.python.eager.backprop' has no attribute '_record_gradient'
    
    Call arguments received:
      • inputs=tf.Tensor(shape=(200, 28, 28, 1), dtype=float32)
